In [1]:
import numpy as np
from datetime import datetime, timedelta
import random
from time import time

In [2]:
# Generate the list of dates
st = datetime.strptime('01/01/2024', '%d/%m/%Y')
n = 365 * 10
list_of_dates = [st + timedelta(days=i) for i in range(n) if np.random.uniform() > 0.8]
random.shuffle(list_of_dates)

In [3]:
def find_date_pairs_brute_force(dates):
    """Finds all pairs of dates in the list that are exactly 7 days apart using a brute force approach.

    Args:
    - dates (list): A list of datetime objects
    
    Returns:
    - list: A list of tuples, where each tuple contains two dates that are 7 days apart
    """
    pairs = []
    
    # Loop through each date
    for i in range(len(dates)):
        first_year, first_month, first_date = split_date_into_components(dates[i])
        first_days_since_start = date_to_ordinal(first_year, first_month, first_date)
        
        for j in range(i + 1, len(dates)):
            sec_year, sec_month, sec_date = split_date_into_components(dates[j])
            sec_days_since_start = date_to_ordinal(sec_year, sec_month, sec_date)
            
            # Check if dates are exactly 7 days apart
            if abs(sec_days_since_start - first_days_since_start) == 7:
                pairs.append((dates[i], dates[j]))
    
    return pairs

In [4]:
def split_date_into_components(date):
    """Splits a given datetime object into its year, month, and day components.
    
    This function parses the datetime object into a string before splitting it into its components.
    
    Args:
    - date (datetime): A datetime object
    
    Returns:
    - tuple: Three integers representing year, month, and day
    
    Note:
    - The datetime object is first converted into a string before being split into its individual 
      components. This approach is intentionally used to avoid performing any further operations 
      involving datetime objects later in the process.
    """
    date_str = str(date)
    date_component = date_str.split(" ")[0].split("-")
    
    return int(date_component[0]), int(date_component[1]), int(date_component[2])

def is_leap_year(year):
    """Determines if the given year is a leap year.
    
    Args:
    - year(int): The year to check
    
    Returns:
    - bool: True if the year is a leap year, False otherwise
    """
    return (year % 4 == 0 and year % 100 != 0) or (year % 400 == 0)

def date_to_ordinal(year, month, day):
    """Converts a date into its ordinal value (number of days since year 1)

    Args:
    - year (int): The year component of the date.
    - month (int): The month component of the date.
    - day (int): The day component of the date.
    
    Returns:
    - int: The ordinal value representing the number of days since year 1
    
    Note:
    - Accounts for leap years and varying month lengths
    """   
    # Days in each month
    days_in_month = [31, 28 + is_leap_year(year), 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    
    # Calculate the ordinal value
    ordinal = 0
    
    # Add days for complete years
    for y in range(1, year):
        ordinal += 365 + is_leap_year(y)

    # Add days for complete months of the current year
    for m in range(1, month):
        ordinal += days_in_month[m - 1]

    # Add the days of the current month
    ordinal += day

    return ordinal

def find_date_pairs_optimized(dates):
    """Finds all pairs of dates in the list that are exactly 7 days apart using an optimized approach.

    This method uses a dictionary to store the ordinal values of dates, allowing for faster lookup of
    date pairs.

    Args:
    - dates (list): A list of datetime objects.
    
    Returns:
    - list: A list of tuples, where each tuple contains two dates that are 7 days apart.
    """
    
    # Dictionary to store dates as keys
    date_dict, pairs = {}, []
    
    for date in dates:
        year, month, day = split_date_into_components(date)
        
        # Convert the date to its ordinal value
        days_since_start = date_to_ordinal(year, month, day)
   
        # Check if there's a date exactly 7 days before
        if days_since_start - 7 in date_dict:
            pairs.append((date_dict[days_since_start - 7], date))
            
        # Check if there's a date exactly 7 days after
        if days_since_start + 7 in date_dict:
            pairs.append((date, date_dict[days_since_start + 7]))
        
        # Add this date to the dictionary
        date_dict[days_since_start] = date
    
    return pairs

In [5]:
%%time

# Measure the runtime of the brute force solution
start = time()
pairs_brute_force = find_date_pairs_brute_force(list_of_dates)
end = time()

print(f"Brute force solution runtime: {end - start:.6f} seconds")
print(f"Number of pairs found: {len(pairs_brute_force)}")

Brute force solution runtime: 95.160221 seconds
Number of pairs found: 150
CPU times: total: 1min 34s
Wall time: 1min 35s


In [6]:
%%time

# Measure the runtime of the optimized solution
start = time()
pairs_optimized = find_date_pairs_optimized(list_of_dates)
end = time()

print(f"Optimized solution runtime: {end - start:.6f} seconds")
print(f"Number of pairs found: {len(pairs_optimized)}")

Optimized solution runtime: 0.258991 seconds
Number of pairs found: 150
CPU times: total: 188 ms
Wall time: 259 ms


## Validation of Pair Matching Between Optimized and Brute Force Methods


In [7]:
sorted_pairs_brute_force = []

# Sort the individual date pairs from the brute force method and store them in a list
for pairs in pairs_brute_force:
    sorted_pairs_brute_force.append(tuple(sorted(pairs)))

# Sort the entire list of pairs from the brute force method
sorted_pairs_brute_force = sorted(sorted_pairs_brute_force)

In [8]:
sorted_pairs_optimized = []

# Sort the individual date pairs from the optimized method and store them in a list
for pairs in pairs_optimized:
    sorted_pairs_optimized.append(tuple(sorted(pairs)))

# Sort the entire list of pairs from the optimized method
sorted_pairs_optimized = sorted(sorted_pairs_optimized)

In [9]:
# Validate that both methods (optimized and brute force) yield the same results
try:
    # Compare sets of sorted pairs from both methods; if they match, the methods are equivalent.
    assert set(sorted_pairs_optimized) == set(sorted_pairs_brute_force)
    print("Both dataframes match")
    
# Handle exceptions if the results differ between the methods
except AssertionError:
    # Find pairs present in optimized results but missing from brute force results.
    missing_from_brute_force = set(sorted_pairs_optimized) - set(sorted_pairs_brute_force)
    
    # Print how many pairs are missing from brute force method, if any
    if missing_from_brute_force:
        print("Missing from brute force:", len(missing_from_brute_force))
        
    # Find pairs present in brute force results but missing from optimized results.
    missing_from_optimized = set(sorted_pairs_brute_force) - set(sorted_pairs_optimized)
    
    # Print how many pairs are missing from optimized method, if any
    if missing_from_optimized:
        print("Missing from optimized:", len(missing_from_optimized))

Both dataframes match
